# Mineração de Dados (Análise de Tendências Temporais e Sazonalidade)
Agora, vamos usar os atributos de tempo (`Ano`, `Mes`) para analisar o desempenho de vendas ao longo do tempo.

O foco é a **agregação** das vendas por período (mês e semana) para detectar **tendências** de crescimento/queda e **padrões sazonais** (picos em certos meses).

### 1. Carregar o dataset tratado

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Carregar o dataset tratado (pois contém as colunas de tempo criadas anteriormente: Ano, Mes, Semana_Ano)
try:
    dados_tempo = pd.read_excel("dataset_tratado.xlsx")
    # Reconvertendo Data para datetime (necessário após carregar CSV)
    dados_tempo["Data"] = pd.to_datetime(dados_tempo["Data"])
    print("Dataset tratado carregado para Análise Temporal.")
except FileNotFoundError:
    print("ERRO: Certifique-se de que o arquivo 'dataset_tratado.excel' está na mesma pasta.")
    exit()

print("-" * 50)
print("MINERAÇÃO 4.2: ANÁLISE DE TENDÊNCIAS E SAZONALIDADE")
print("-" * 50)

# Criar as colunas de tempo novamente, caso não existam no CSV salvo
dados_tempo['Ano'] = dados_tempo['Data'].dt.year
dados_tempo['Mes'] = dados_tempo['Data'].dt.month

### 2. Agregação: Total de Receita e Quantidade Vendida por Ano e Mês
A) ANÁLISE DE TENDÊNCIA MENSAL

In [ ]:
print("A) Analisando Tendência de Vendas por Mês...")
 
vendas_mensais = dados_tempo.groupby(['Ano', 'Mes']).agg(
    Receita_Total=('Total de Vendas', 'sum'),
    Qtd_Total=('Quantidade Vendida', 'sum')
).reset_index()

### Criar uma coluna 'Periodo' para o gráfico (usando o formato AAAA-MM)

In [ ]:
vendas_mensais['Periodo'] = vendas_mensais['Ano'].astype(str) + '-' + vendas_mensais['Mes'].astype(str).str.zfill(2)

print("\nTendência Mensal (Top 5 Meses):")
print(vendas_mensais.sort_values(by='Receita_Total', ascending=False).head().to_markdown(index=False))

### 3. Visualização da Série Temporal da Receita

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='Periodo', y='Receita_Total', data=vendas_mensais, marker='o')
plt.title('Receita Total ao Longo do Tempo (Tendência Mensal)')
plt.xlabel('Mês/Ano')
plt.ylabel('Receita Total')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

### 4. Agregação: Receita Média por Dia da Semana

B) ANÁLISE DE SAZONALIDADE SEMANAL


In [ ]:
# Dia_Semana_Nome será criado aqui, se não estiver no CSV
dados_tempo['Dia_Semana_Nome'] = dados_tempo['Data'].dt.day_name(locale='pt_BR')
# Mapeamento para garantir a ordem correta no gráfico
ordem_dias = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']

vendas_por_dia = dados_tempo.groupby('Dia_Semana_Nome')['Total de Vendas'].mean().reindex(ordem_dias).reset_index()
vendas_por_dia.columns = ['Dia_Semana', 'Receita_Media']

print("\nReceita Média por Dia da Semana:")
print(vendas_por_dia.round(2).to_markdown(index=False))

# 2. Visualização da Sazonalidade Semanal
plt.figure(figsize=(8, 5))
sns.barplot(x='Dia_Semana', y='Receita_Media', data=vendas_por_dia, palette='viridis')
plt.title('Receita Média por Dia da Semana')
plt.xlabel('Dia da Semana')
plt.ylabel('Receita Média (R$)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()